<a href="https://colab.research.google.com/github/Bhagavanth67/HealthIntel-CLI/blob/main/7_Final__HealthIntel_CLI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import csv, os, pandas as pd, matplotlib.pyplot as plt
from datetime import datetime
from google.colab import files


FILENAME = "healthintel_data.csv"

# ---------------------- Input Validation ---------------------- #
def get_valid_str(msg):
    while True:
        s = input(msg).strip()
        if s:
            return s
        print("Input cannot be empty.")

def get_valid_int(msg, min_v=0):
    while True:
        try:
            v = int(input(msg))
            if v > min_v:
                return v
            print(f"Enter a number greater than {min_v}.")
        except ValueError:
            print("Invalid integer input.")

def get_valid_float(msg, min_v=0):
    while True:
        try:
            v = float(input(msg))
            if v > min_v:
                return v
            print(f"Enter a number greater than {min_v}.")
        except ValueError:
            print("Invalid numeric input.")


# ---------------------- File Setup ---------------------- #
def ensure_csv():
    """Ensure CSV file exists with headers."""
    if not os.path.exists(FILENAME):
        with open(FILENAME, "w", newline="") as f:
            writer = csv.writer(f)
            writer.writerow(["DateTime","Name","Age","Height (m)","Weight (kg)","BMI","Category"])


# ---------------------- Core Features ---------------------- #
def add_record():
    """Add a new BMI entry (preserves history)."""
    ensure_csv()
    name = get_valid_str("Enter your name: ")
    age = get_valid_int("Enter your age: ", 0)
    height = get_valid_float("Enter your height (in metres): ", 0.5)
    weight = get_valid_float("Enter your weight (in kilograms): ", 1.0)

    bmi = round(weight / (height ** 2), 2)
    if bmi < 18.5:
        category = "Underweight"
    elif bmi < 24.9:
        category = "Healthy weight"
    elif bmi < 29.9:
        category = "Overweight"
    else:
        category = "Obese"

    timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")

    # Append a new record instead of overwriting
    with open(FILENAME, "a", newline="") as f:
        writer = csv.writer(f)
        if os.stat(FILENAME).st_size == 0:
            writer.writerow(["DateTime","Name","Age","Height (m)","Weight (kg)","BMI","Category"])
        writer.writerow([timestamp,name,age,height,weight,bmi,category])

    print(f"\n✅ Record added successfully for {name}.")
    print(f"BMI: {bmi} | Category: {category}\n")


def view_records():
    """Display all records."""
    if not os.path.exists(FILENAME):
        print("No records found.")
        return
    df = pd.read_csv(FILENAME)
    if df.empty:
        print("No data available yet.")
        return
    print("\n=== All Recorded Users ===")
    print(df.to_string(index=False))


def analyze_data():
    """Show data analytics."""
    if not os.path.exists(FILENAME):
        print("No data available.")
        return
    df = pd.read_csv(FILENAME)
    if df.empty:
        print("No data for analysis.")
        return

    total = len(df)
    avg_bmi = round(df["BMI"].mean(), 2)
    counts = df["Category"].value_counts().to_dict()

    print("\n=== Health Statistics ===")
    print(f"Total Records: {total}")
    print(f"Average BMI: {avg_bmi}")
    print("Category Distribution:")
    for c, v in counts.items():
        print(f"  {c}: {v}")


def visualize_data():
    """Visualize BMI data (category bar + user-specific trend)."""
    if not os.path.exists(FILENAME):
        print("No data to visualize.")
        return
    df = pd.read_csv(FILENAME)
    if df.empty:
        print("No data for visualization.")
        return

    # --- Bar Chart: Category Distribution ---
    plt.figure(figsize=(6,4))
    df["Category"].value_counts().plot(kind="bar", color="cornflowerblue", edgecolor="black")
    plt.title("BMI Category Distribution")
    plt.xlabel("Category")
    plt.ylabel("Number of Users")
    plt.grid(axis="y", linestyle="--", alpha=0.6)
    plt.tight_layout()
    plt.show()

    # --- Line Chart: BMI Trend for Selected User ---
    user = input("\nEnter your name to view BMI trend: ").strip().lower()
    df["DateTime"] = pd.to_datetime(df["DateTime"], errors="coerce")
    user_df = df[df["Name"].str.lower() == user].sort_values(by="DateTime")

    if user_df.empty:
        print("No records found for that user.")
        return

    plt.figure(figsize=(7,4))
    plt.plot(user_df["DateTime"], user_df["BMI"], marker="o", linestyle="-", color="seagreen")
    plt.title(f"BMI Trend Over Time - {user.capitalize()}")
    plt.xlabel("Date & Time")
    plt.ylabel("BMI Value")
    plt.xticks(rotation=45)
    plt.grid(True, linestyle="--", alpha=0.6)
    plt.tight_layout()
    plt.show()


def export_csv():
    """Download CSV to local machine (Colab)."""
    if os.path.exists(FILENAME):
        print("\nDownloading latest health data file...")
        files.download(FILENAME)
    else:
        print("No CSV file found to download.")


# ---------------------- Menu System ---------------------- #
def main_menu():
    ensure_csv()
    while True:
        print("\n======= HealthIntel CLI =======")
        print("1. Add Record")
        print("2. View Records")
        print("3. Analyze Data")
        print("4. Visualize Data")
        print("5. Export CSV")
        print("6. Exit")

        choice = input("Choose an option (1-6): ").strip()

        if choice == "1":
            add_record()
        elif choice == "2":
            view_records()
        elif choice == "3":
            analyze_data()
        elif choice == "4":
            visualize_data()
        elif choice == "5":
            export_csv()
        elif choice == "6":
            print("\nThank you for using HealthIntel CLI. Stay healthy and consistent!")
            break
        else:
            print("Invalid choice. Please enter a number between 1 and 6.")


# ---------------------- Run Program ---------------------- #
if __name__ == "__main__":
    try:
        main_menu()
    except Exception as e:
        print(f"\n[ERROR] Unexpected issue: {e}")
    finally:
        if os.path.exists(FILENAME):
            print("\nAuto-downloading your final CSV file...")
            files.download(FILENAME)
